In [1]:
#load "sig.fsx"
open CSCI374.ExtraReflection

The below script needs to be able to find the current output cell; this is an easy method to get it.

# F# Type System

## How to build bigger types

- Have various base types like **int bool unit char**

- Ways to build (nested) compound types: *tuples, lists, array*
    - Coming soon: more ways to build compound types
    - First: 3 most important type building blocks in any language

- **Each of**: A **t** value contains values of *each* of **t1 t2 ... tn** (Product)

- **One of**: A **t** value contains values of *one* of **t1 t2 ... tn** (Union)

- **Self reference**: A **t** value can *refer* to other **t** values (Recursive)

**Remarkable:** A lot of data can be described with just these building blocks

*Note: These are not the common names for these concepts*

## Creating new F# types

New types in F# are constructed by combining other types using two basic operations:

- **Product Type**: type **typeW** = **typeX** "times" **typeY**
- **Union Type**: type **typeZ** = **typeX** "plus" **typeY**

## Representing Pairs

![type-pair](img/int-tuple.png)

- pair of ints written **int * int**

## Representing Pairs

![type-record](img/record.png)

## Representing a Choice

![type-choice](img/choice1.png)

## Representing a Choice (cont.)

![type-choice](img/choice2.png)

## Representing a Choice (cont.)

![type-choice](img/choice3.png)

## More Examples

- Tuples build each-of types
    - **int * bool** contains an **int** *and* a **bool**

- Options build one-of types
    - **int option** contains an **int** *or* it contains no data

- Lists use all three building blocks
    - **int list** contains an **int** *and* another **int list** *or* it contains no data

- And of course we can nest compound types
    - **((int \* int) option \* (int list list)) option**


## Records

Record values have fields (any name) holding values

    {n1 = v1; ...; nn = vn}

Record types have fields (and name) holding types

    {n1 : t1; ...; nn : tn}

The order of fields in a record value or type never matters


## Records

A record type is exactly that, a tuple where each element is labeled.

In [37]:
type ComplexNumberTuple = float * float

type ComplexNumberRecord = { real: float; imaginary: float } // use colon in type

A record type has the standard preamble: 

- `type [typename] =` followed by curly braces
- Inside the curly braces is a list of label:
    - type pairs, separated by semicolons
    
**Remember:** all lists in F# use semicolon separators – commas are for tuples

## Record Value

To create a record value:
- use a similar format to the type definition
- but using equals signs after the labels.

This is called a **record expression**.

In [40]:
// Record value
let record = { real = 1.1; imaginary = 2.2 } // use equals!

sgn record, record

Item1,Item2
Value: ComplexNumberRecord,{ real = 1.1 imaginary = 2.2 }


In [41]:
let myComplexNumber = { imaginary = 2.2; real = 1.1 } // use equals in let
myComplexNumber

real,imaginary
1.1,2.2


And to “deconstruct” a record, use the same syntax:

In [42]:
let myComplexNumber = { imaginary = 2.109; real = 1.1 }  // "construct"
let { real=re; imaginary=im } = myComplexNumber          // "deconstruct"

printfn "Re: %f, Im: %f" re im

Re: 1.100000, Im: 2.109000



input.fsx (2,5)-(2,30) typecheck warning The field labels and expected type of this record expression or pattern do not uniquely determine a corresponding record type



If you just need a single property, you can use dot notation rather than pattern matching.

In [43]:
myComplexNumber.real

1.1

In [44]:
myComplexNumber.imaginary

2.109

## By name vs by position

Little difference between (4,7,9) and {f=4,g=7,h=9}
- Tuples a little shorter
- Records a little easier to remember “what is where”
- Generally a matter of taste, but for many (6? 8? 12?) fields, a record is usually a better choice

A common decision for a construct's syntax is whether to refer to things by position (as in tuples) or by some (field) name (as with records)
- A common hybrid is like with Java method arguments (and ML functions as used so far):
    - Caller uses position
    - Callee uses variables
    - Could totally do it differently; some languages have

## Unions

A “strange” (?) and totally awesome (!) way to make **one-of** types:

In [ ]:
type mytype = TwoInts of int * int
            | Str of string
            | Pizza

Datatype binding:

- Adds a new type `mytype` to the environment
- Adds constructors to the environment: `TwoInts`, `Str`, and `Pizza`
- A constructor is (among other things), a function that makes values of the new type (or is a value of the new type):
    - `TwoInts : int * int -> mytype`
    - `Str : string -> mytype`
    - `Pizza : mytype`

## Creating Values

Any value of type `mytype` is made from one of the constructors

The value contains:
- A “tag” for “which constructor” (e.g., TwoInts)
- The corresponding data (e.g., (7,9))

Examples:
- `TwoInts(3+4,5+4)` evaluates to `TwoInts(7,9)`
- `Str(if true then "hi" else "bye")` evaluates to `Str("hi")`
- `Pizza` is a value

In [45]:
let v = TwoInts(3+4,5+4)

sgn v, v

Item1,Item2
Value: TwoInts,"TwoInts (7, 9)"


In [46]:
let v = Str(if false then "hi" else "bye")

sgn v, v

Item1,Item2
Value: Str,"Str ""bye"""


In [47]:
let v = Pizza

sgn v, v

Item1,Item2
Value: _Pizza,Pizza


## Deconstruct Value

F# combines the two aspects of accessing a **one-of** value with pattern matching.
- To deconstruct the values of union types into their basic parts, you **always** use pattern matching.


```fsharp
match x with
| Pizza -> 3
| TwoInts (i1, i2) -> i1+i2
| Str s -> String.length s
```    

- A multi-branch conditional to pick branch based on variant
- Extracts data and binds to variables local to that branch
- Type-checking: all branches must have same type

In [49]:
let f x =
    match x with
    | Pizza -> 3
    | TwoInts (i1, i2) -> i1+i2
    | Str s -> String.length s
    
sgn f

Function: x:mytype -> int

In [50]:
f Pizza

3

In [57]:
f (Str(if true then "hi" else "bye"))

2

In [59]:
f (TwoInts(3+4,5+5))

17

## Examples


Enumerations, including carrying other data

```fsharp
type Suit = Club | Diamond | Heart | Spade
type Card = Jack | Queen | King | Ace | Num of int
type Deck = (Suit * Card) list
```

Alternate ways of identifying real-world things/people

```fsharp
type id = StudentNum of int
        | Name of string
                * (string option)
                * string
```

## Example

In [87]:
type CardType = Visa | Mastercard | AmEx
type CardNumber = string
type AccountNumber = int
type PaymentMethod =
    | Cash
    | Cheque of AccountNumber
    | Card of CardType * CardNumber

In [88]:
Card (Mastercard, "734687346")

Item1,Item2
Mastercard,734687346


In [89]:
let printPayment method =
    match method with
    | Cash ->
        printfn "Paid in cash"
    | Cheque checkNo ->
        printfn "Paid by cheque: %i" checkNo
    | Card (cardType,cardNo) ->
        printfn "Paid with %A %A" cardType cardNo
sgn printPayment

Function: method:PaymentMethod -> unit

In [90]:
sgn Cash |> printfn "%A"
printPayment Cash

"Value: _Cash"
Paid in cash


In [93]:
sgn (Cheque 1000) |> printfn "%A"
printPayment (Cheque 1300)

"Value: Cheque"
Paid by cheque: 1300


In [92]:
sgn (Card (Visa, "V12345")) |> printfn "%A"
printPayment (Card (Visa, "V12345"))

"Value: Card"
Paid with Visa "V12345"


## Expression Trees

A more exciting (?) example of a data type, using self-reference

In [94]:
type Exp = Constant of int 
         | Negate of Exp 
         | Add of Exp * Exp
         | Multiply of Exp * Exp
         
let example_exp = Add (Constant 19, Negate (Constant 4))
sgn example_exp, example_exp

Item1,Item2
Value: Add,"Add (Constant 19, Negate (Constant 4))"


How to picture the resulting value in your head:
```
       Add
     /     \
Constant Negate
   |        |
  19    Constant
            |
            4
```

## Recursion

Not surprising:
- Functions over recursive data types are usually recursive

In [97]:
let rec eval e =
    match e with
    | Constant i      -> i
    | Negate e2       -> -(eval e2)
    | Add(e1,e2)      -> (eval e1) + (eval e2)
    | Multiply(e1,e2) -> (eval e1) * (eval e2)

In [95]:
sgn eval, example_exp

Item1,Item2
Function: e:Exp -> int,"Add (Constant 19, Negate (Constant 4))"


In [98]:
eval example_exp

15

In [99]:
let rec number_of_adds e =
    match e with
    | Constant i      -> 0
    | Negate e2       -> number_of_adds e2
    | Add(e1,e2)      -> 1 + number_of_adds e1 + number_of_adds e2
    | Multiply(e1,e2) -> number_of_adds e1 + number_of_adds e2

In [100]:
let example_addcount = Add ((Negate (Add (Constant 1, Constant 32))), Multiply(example_exp, example_exp))
example_addcount, eval example_addcount

Item1,Item2
"Add (Negate (Add (Constant 1, Constant 32)), Multiply (Add (Constant 19, Negate (Constant 4)), Add (Constant 19, Negate (Constant 4))))",192


In [101]:
number_of_adds example_addcount

4

Let's define `max_constant : Exp -> int`

Good example of combining several topics as we program:
- Case expressions
- Local helper functions
- Avoiding repeated recursion
- Simpler solution by using library functions

In [106]:
let rec max_constant e =
    let max_of_two (e1,e2) =
        let m1 = max_constant e1
        let m2 = max_constant e2
        if m1 > m2 then m1 else m2 

    match e with
    | Constant i      -> i
    | Negate e2       -> max_constant e2
    | Add(e1,e2)      -> max_of_two (e1,e2)
    | Multiply(e1,e2) -> max_of_two (e1,e2)

In [107]:
max_constant example_addcount // 32

32

## Parameterization

- Both *union* and *record* types can be parameterized.

- Parameterizing a type means leaving *one or more* of the types within the type being defined to be determined later by the consumer of the types.
  - When defining types, you must be a little bit more explicit about which types are variable.
  
- F# supports two syntaxes for type parameterization.

First, you place the type being parameterized between the keyword `type` and the name of the type, as follows:
- The names of type parameters always start with a single quote (') followed by an alphanumeric name for the type.
- Typically, just a single letter is used.
- If multiple parameterized types are required, you separate them with commas.

In [108]:
type 'a BinaryTree =
    | BinaryNode of 'a BinaryTree * 'a BinaryTree
    | BinaryValue of 'a

In [109]:
let intbt = 
    BinaryNode(
        BinaryNode ( BinaryValue 1, BinaryValue 2),
        BinaryNode ( BinaryValue 3, BinaryValue 4)
    )
sgn intbt

Value: BinaryTree<int>

In [112]:
let chbt = 
    BinaryNode(
        BinaryNode ( BinaryValue 'a', BinaryValue 'b'),
        BinaryValue ('c')
    )
sgn chbt

Value: BinaryTree<char>

In the second syntax, you place the types being parameterized in *angle brackets* after the type name, as follows:

In [113]:
type Tree<'a> =
    | Node of Tree<'a> list
    | Value of 'a

In [114]:
let tree1 =
    Node(
        [ Node( [Value "one"; Value "two"] ) ;
          Node( [Value "three"; Value "four"] ) ]
    )
sgn tree1, tree1

Item1,Item2
Value: Tree<string>,"Node [Node [Value ""one""; Value ""two""]; Node [Value ""three""; Value ""four""]]"


In [115]:
let tree2 = 
    Node(
        [ Node( [Value 1; Value 2] ) ;
          Node( [Value 3] ) ]
    )
sgn tree2, tree2

Item1,Item2
Value: Tree<int>,Node [Node [Value 1; Value 2]; Node [Value 3]]


The syntax for creating and consuming an instance of a parameterized type does not change from that of creating and consuming a nonparameterized type.
- This is because the compiler will automatically infer the type parameters of the parameterized type.

In [116]:
let rec printTreeValues x =
    match x with
    | Node l -> List.iter printTreeValues l
    | Value x -> printf "%A, " x

In [117]:
printTreeValues tree1

"one", "two", "three", "four", 

In [118]:
printTreeValues tree2

1, 2, 3, 

In [138]:
let rec sum t = 
    match t with
    | Value i -> i
    | Node [] -> 0 // ??? string or int ???
    | Node (x::xs) -> (sum x) + (sum (Node xs))
    
sgn sum

Function: t:Tree<int> -> int

In [136]:
tree2, sum tree2

Item1,Item2
Node [Node [Value 1; Value 2]; Node [Value 3]],6


In [137]:
tree1, sum tree1

Stopped due to error

input.fsx (1,12)-(1,17) typecheck error Type mismatch. Expecting a
    'Tree<int>'    
but given a
    'Tree<string>'    
The type 'int' does not match the type 'string'



Cell not executed: compilation error

## Polymorphism

A **generic** or **polymorphic** subprogram takes parameters of different types on different activations
- Overloaded subprograms provide **ad hoc polymorphism**
- **Subtype polymorphism** means that a variable of type `T` can access any object of type `T` or any type derived from `T` (OOP languages)

## Parametric Polymorphism

**Parametric** polymorphism is provided by a subprogram that takes **generic** parameters that are used in type expressions that describe the types
of the parameters of the subprogram.

Different instantiations of such subprograms can be given different generic parameters, producing subprograms that take different types of parameters.
 - A cheap compile-time substitute for dynamic binding

## Generic Functions

A key feature of F# is the automatic generalization of code. The combination of automatic generalization and type inference makes:
- many programs simpler, and more general,
- enhances code reuse

Automatic generalization is applied when a `let` or `member` definition doesn't fully constrain the types of inputs or outputs.
- You can tell automatic generalization has been applied by the presence of type variables in an inferred type, and
- ultimately by the fact that you can reuse a construct with multiple types.

In [139]:
// Evaluate functions in the interpreter
let getFirst (a,b,c) = a
let mapPair f g (x, y) = (f x, g y)
sgn getFirst, sgn mapPair

Item1,Item2
Function: tupledArg:obj * obj * obj -> obj,Function: f:(obj -> obj) -> g:(obj -> obj) -> tupledArg:obj * obj -> obj * obj


In [140]:
getFirst (1,2.0,3)

1

In [141]:
getFirst ('a',2.0,3)

a

In [144]:
mapPair (fun x-> int (x+1.5)) (fun x-> string (x*2)) (1.4,3)

Item1,Item2
2,6


## Units

F# has the ability to define units of measure and associate them with other types.
- The unit of measure is then "attached" to the declared type as a additional type and prevents mixing different types.

In [145]:
[<Measure>] 
type cm

[<Measure>] 
type inch

let meter = 100<cm>
let yard = 36<inch>

In [146]:
yard + meter // not allowed: different measures

Stopped due to error

input.fsx (1,8)-(1,13) typecheck error The unit of measure 'cm' does not match the unit of measure 'inch'

input.fsx (1,6)-(1,7) typecheck error The unit of measure 'cm' does not match the unit of measure 'inch'



Cell not executed: compilation error

In [147]:
yard + 1  // not allowed: didn't specify a measure type

Stopped due to error

input.fsx (1,8)-(1,9) typecheck error The type 'int' does not match the type 'int<inch>'

input.fsx (1,6)-(1,7) typecheck error The type 'int' does not match the type 'int<inch>'



Cell not executed: compilation error

In [149]:
yard + 1<inch>, meter + 11<cm>

Item1,Item2
37,111


## Exceptions

- F# supports throwing and catching exceptions.

- Exception definitions in F# are similar to defining a constructor of a union type, and the syntax for handling exceptions is similar to pattern matching.

- You define exceptions using the `exception` keyword followed:
  - by the **name** of the exception, and
  - then optionally the keyword of and the types of any values the exception should contain, with multiple types separated by asterisks.
 

In [ ]:
exception WrongSecond of int

## Raising Exceptions

You can raise exceptions with the `raise` keyword, as shown in the `else` clause in the following `testSecond` function.

In [ ]:
let primes = [ 2; 3; 5; 7; 11; 13; 17; 19; 23; 29; 31; 37; 41; 43; 47; 53; 59 ]

let testSecond() =
    try
        let currentSecond = System.DateTime.Now.Second in
        if List.exists (fun x -> x = currentSecond) primes then
            failwithf "A prime second: %d" currentSecond
        else
            raise (WrongSecond currentSecond)
    with
        WrongSecond x -> printfn "The current was %i, which is not prime" x
    printfn "Done!!!"
    
testSecond()

F# also has an alterative to the `raise` keyword, the `failwith` function
- If you just want to raise an exception with a text description of what went wrong, you can use `failwith` to raise a generic exception that contains the text passed to the function.

## Exception Handling

The `try` and `with` keywords handle exceptions.
- The expressions that are subject to error handling go between the `try` and `with` keywords, and
- one or more pattern matching rules must follow the `with` keyword.

When trying to match an F# exception, the syntax follows that of trying to match an F# constructor from a union type.
- The first half of the rule consists of the exception name, followed by identifiers or wildcards to match values that the exception contains.
- The second half of the rule is an expression that states how the exception should be handled.